<a href="https://colab.research.google.com/github/cape2021/Team2/blob/main/ServeModelWeb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Use of Tensor flow 1.X is recomended for the current notebook
%tensorflow_version 1.x #If you are using Google Colab

In [ ]:
%pip install Google-Colab-Transfer #If you are Using Google Colab 

In [ ]:
import colab_transfer

gd = colab_transfer.get_path_to_home_of_google_drive()
lm = colab_transfer.get_path_to_home_of_local_machine()

colab_transfer.mount_google_drive()

In [ ]:
!git clone https://github.com/NVlabs/stylegan2-ada.git

In [ ]:
!pip install flask-ngrok

In [ ]:
import sys

# load stylegan directory
sys.path.insert(0, "/content/stylegan2-ada")

import threading
import os
import pickle
import numpy as np
import PIL.Image
from IPython.display import Image
import matplotlib.pyplot as plt
import dnnlib
import dnnlib.tflib as tflib


class getSeedImg:
  def __init__(self, seed):
    self.seed = seed
    self.run()

  def seed2vec(self,Gs, seed):
    print(seed)
    rnd = np.random.RandomState(seed)
    return rnd.randn(1, *Gs.input_shape[1:])

  def init_random_state(self,Gs, seed):
    rnd = np.random.RandomState(seed) 
    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
    tflib.set_vars({var: rnd.randn(*var.shape.as_list()) for var in noise_vars}) # [height, width]

  def display_image(self,image):
    plt.axis('off')
    plt.imshow(image)
    plt.show()

  def generate_image(self,Gs, z, truncation_psi):
    # Render images for dlatents initialized from random seeds.
      Gs_kwargs = {
          'output_transform': dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True),
          'randomize_noise': False
      }
      if truncation_psi is not None:
        Gs_kwargs['truncation_psi'] = truncation_psi

      label = np.zeros([1] + Gs.input_shapes[1][1:])
      images = Gs.run(z, label, **Gs_kwargs) # [minibatch, height, width, channel]
      return images[0]

  def run(self):
    tflib.init_tf()
    # Specify your pickle file to generate images
    with open("/content/drive/MyDrive/FACES/Train/00001-facedataset-auto_no_ramp1-kimg10000-p0.713-resumecustom-freezed10/network-snapshot-000314.pkl", "rb") as fp:
      _G, _D, Gs = pickle.load(fp)
    if os.path.isfile('./mood.png'):
      os.remove('mood.png')
    self.init_random_state(Gs, 10)
    z = self.seed2vec(Gs, self.seed)
    img = self.generate_image(Gs, z, 1.0)
    PIL.Image.fromarray(img, 'RGB').save(f'./mood.png')



#getSeedImg(1)
#th = threading.Thread(target=getSeedImg, args=(0,))
#th.start()
#th.join()


In [ ]:
from flask import Flask
from flask import send_file
from flask_ngrok import run_with_ngrok


app = Flask(__name__)
run_with_ngrok(app)  


@app.route('/<path:text>', methods=['GET', 'POST'])
def all_routes(text):
    if text.isnumeric():
      th = threading.Thread(target=getSeedImg, args=(int(text),))
      th.start()
      th.join()
      filename = 'mood.png'
      return send_file(filename, mimetype='image/png')
    #else:
      #return redirect(url_for('404_error'))


if __name__ == '__main__':
  app.run()